In [1]:
import numpy
import cv2
import matplotlib.pyplot as plt

import glob
import os
from pprint import pprint
from pathlib import Path
import xml.etree.ElementTree as ET


from IPython.display import Image
from IPython.core.display import HTML

### mAP

Bounding boxes are a crude approximation of many objects. 
Ground truth boxes :  Box that denote the true position of an object (x,y,width,height).

### IoU (Intersection over Union)
It is the measure of the ratio between the intersection and the union of the predicted box and the ground truth boxes.[1]


In [2]:
Image(url= "https://tarangshah.com/blog/images/map-7.png", width=300, height=300)

### Precision & Recall

True Positive
False Positive
True Negative
False Negative

Total Number of Objects

Precision : 
Recall :

In [14]:
def read_annotation(annotation_path:str):
    status = True
    
    tree = ET.parse(annotation_path)
    root = tree.getroot()

    all_results = []
    for boxes in root.iter('object'):
        single_result = {}

        filename = root.find('filename').text
        class_name = boxes.find('name').text
        
        ymin, xmin, ymax, xmax = None, None, None, None

        for box in boxes.findall("bndbox"):
            ymin = int(box.find("ymin").text)
            xmin = int(box.find("xmin").text)
            ymax = int(box.find("ymax").text)
            xmax = int(box.find("xmax").text)

        list_with_single_boxes = [xmin, ymin, xmax, ymax]
        single_result['label'] = class_name
        single_result['bbox'] = list_with_single_boxes
        
        all_results.append(single_result)

    result = {}
    result["status"] = status 
    result["results"] = all_results 
    
    return result

result = read_annotation("/home/jayasimha/Pictures/door_2.xml")
pprint(result)

{'results': [{'bbox': [80, 40, 144, 185], 'label': 'door'},
             {'bbox': [34, 6, 67, 55], 'label': 'lamp'}],
 'status': True}


In [21]:
class ObjDetector():
    def __init__(self, ):
        self.graph_def = None
        self.out = None
    
    def initialize_model(pb_path : str):
        with tf.gfile.FastGFile(pb_path, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
    
    def run_detection(img_path :str):
        with tf.Session() as sess:
            # Restore session
            sess.graph.as_default()
            tf.import_graph_def(self.graph_def, name='')

            # Read and preprocess an image.
            img = cv.imread(img_path)
            img = cv.resize(img,(480,640))
    
            rows = img.shape[0]
            cols = img.shape[1]
            inp = cv.resize(img, (300, 300))
            inp = inp[:, :, [2, 1, 0]]  # BGR2RGB

            # Run the model
            out = sess.run([sess.graph.get_tensor_by_name('num_detections:0'),
                            sess.graph.get_tensor_by_name('detection_scores:0'),
                            sess.graph.get_tensor_by_name('detection_boxes:0'),
                            sess.graph.get_tensor_by_name('detection_classes:0')],
                           feed_dict={'image_tensor:0': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})

                
    
    def get_result():
        # Visualize detected bounding boxes.
        num_detections = int(out[0][0])
        for i in range(num_detections):
            classId = int(out[3][0][i])
            score = float(out[1][0][i])
            bbox = [float(v) for v in out[2][0][i]]
            if score > 0.3:
                x = bbox[1] * cols
                y = bbox[0] * rows
                right = bbox[3] * cols
                bottom = bbox[2] * rows
                cv.rectangle(img, (int(x), int(y)), (int(right), int(bottom)), (125, 255, 51), thickness=2)    

In [20]:
def detect_object(img_path :str):
    status = False
    
    img = cv2.imread(img_path)
    
    result = {}
    result["status"] = status 
    result["results"] = []
    
    return result

In [16]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [17]:
def find_accuracy(dataset_path:str):
    
    img_file_list = glob.glob( dataset_path + "/*.jpg", recursive=True)
    
    for img in img_file_list :
        name = Path(img).stem
        img_path = dataset_path + name + ".jpg"
        annotation_path = dataset_path + name + ".xml"

        print(img_path,annotation_path)
        
        detection_result = detect_object(img)
        annotation_result = read_annotation(annotation_path)
        
        bb_1 = detection_result["box"]
        bb_2 = annotation_result["box"]
        
        obj_IoU = bb_intersection_over_union(bb_1,bb_2)
        
        

In [18]:
find_accuracy("/home/jayasimha/Pictures/")

/home/jayasimha/Pictures/door.jpg /home/jayasimha/Pictures/door.xml


TypeError: detect_object() missing 1 required positional argument: 'label'

### References

1. https://tarangshah.com/blog/2018-01-27/what-is-map-understanding-the-statistic-of-choice-for-comparing-object-detection-models/
2. https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/
3. http://homepages.inf.ed.ac.uk/ckiw/postscript/ijcv_voc09.pdf
4. http://cocodataset.org/#detection-eval
5. https://stackoverflow.com/questions/53317592/reading-pascal-voc-annotations-in-python